\newpage

%%latex
\hypersetup{linkcolor=black}

## Appendix

In [ ]:
%pip install python-dotenv
%pip install googlemaps
%pip install cvxpy
%pip install cvxopt
%pip install gurobipy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 6.4 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.
  Using cached osmnx-2.0.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached geopandas-1.0.1-py3-none-any.whl.metadata (2.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 9.8 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 2.8 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packag

In [2]:
import os
import time
import googlemaps
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import cvxpy as cp

# Load the .env file
load_dotenv()

# Initialize Google Maps API
API_KEY = os.getenv("API_KEY")
gmaps = googlemaps.Client(key=API_KEY)


def get_distance_matrix(gmaps, coordinates, mode="driving", buffer_minutes=5):
    """
    Fetch a distance matrix using Google Maps API, handling specific requirements for transit mode.

    Args:
        gmaps (googlemaps.Client): Google Maps API client.
        coordinates (list): List of coordinates (latitude, longitude) for locations.
        mode (str): Transportation mode ("driving", "transit", or "walking").
        buffer_minutes (int): Number of minutes to add as a buffer to the departure time for transit.

    Returns:
        np.ndarray: Distance matrix with travel times in seconds.
    """
    # Set departure time to now + buffer_minutes for transit mode
    departure_time = int(time.time()) + buffer_minutes * 60 if mode == "transit" else "now"

    n = len(coordinates)
    matrix = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            if i == j:
                matrix[i][j] = 0  # Zero time for same location
                continue

            # Try to fetch time for the given mode
            try:
                result = gmaps.distance_matrix(
                    origins=[coordinates[i]],
                    destinations=[coordinates[j]],
                    mode=mode,
                    departure_time=departure_time if mode == "transit" else None,
                )
                element = result["rows"][0]["elements"][0]

                if element["status"] == "OK":
                    matrix[i][j] = element["duration"]["value"]  # Travel time in seconds
                else:
                    raise Exception(f"{mode.capitalize()} not available")
            except Exception as e:
                # Handle transit-specific fallback to walking
                if mode == "transit":
                    print(f"Transit unavailable between location {i} and {j}. Falling back to walking.")
                    try:
                        result = gmaps.distance_matrix(
                            origins=[coordinates[i]],
                            destinations=[coordinates[j]],
                            mode="walking",
                        )
                        element = result["rows"][0]["elements"][0]
                        if element["status"] == "OK":
                            matrix[i][j] = element["duration"]["value"]  # Walking time in seconds
                        else:
                            matrix[i][j] = np.inf  # No valid route
                    except:
                        matrix[i][j] = np.inf  # No valid route
                else:
                    matrix[i][j] = np.inf  # Assign a large value if no route is found

    return matrix




def solve_tsp(distance_matrix):
    """
    Solve the Traveling Salesman Problem (TSP) using CVXPY with MTZ constraints.

    Args:
        distance_matrix (np.ndarray): Matrix of travel times.

    Returns:
        tuple: (Tour matrix indicating optimal paths, optimal cost).
    """
    n = distance_matrix.shape[0]
    x = cp.Variable((n, n), boolean=True)
    u = cp.Variable(n)

    # Objective: Minimize total travel time
    objective = cp.Minimize(cp.sum(cp.multiply(distance_matrix, x)))

    # Constraints
    constraints = []

    # Each location must be visited exactly once
    constraints += [cp.sum(x, axis=0) == 1]
    constraints += [cp.sum(x, axis=1) == 1]

    # Avoid self-loops
    for i in range(n):
        constraints.append(x[i, i] == 0)

    # Subtour elimination (MTZ constraints)
    for i in range(1, n):
        for j in range(1, n):
            if i != j:
                constraints.append(u[i] - u[j] + n * x[i, j] <= n - 1)

    for i in range(1, n):
        constraints.append(u[i] >= 2)
        constraints.append(u[i] <= n)

    # Solve the problem
    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.GLPK_MI)

    # Extract the solution
    tour_matrix = np.round(x.value)
    return tour_matrix, problem.value


def extract_tour(tour_matrix):
    """
    Extract the optimal tour from the tour matrix.

    Args:
        tour_matrix (np.ndarray): Matrix indicating the optimal paths.

    Returns:
        list: Sequence of indices representing the optimal route.
    """
    n = len(tour_matrix)
    route = []
    current = 0
    while len(route) < n:
        route.append(current)
        next_step = np.argmax(tour_matrix[current])
        tour_matrix[current] = 0  # Mark as visited
        current = next_step
    route.append(route[0])  # Return to start
    return route


def pretty_print_route(optimal_route, total_time):
    """
    Pretty print the optimal route in a user-friendly format, including total time.

    Args:
        optimal_route (list): List of location names representing the optimal route.
        total_time (float): Total travel time in seconds.

    Returns:
        str: Formatted string describing the optimal route with total travel time.
    """
    # Convert total time from seconds to hours, minutes, and seconds
    hours, remainder = divmod(int(total_time), 3600)
    minutes, seconds = divmod(remainder, 60)

    # Format route description
    route_description = " -> ".join(optimal_route)

    # Return formatted output
    return f"Total time: {hours} hrs, {minutes} mins, {seconds} seconds\nThe optimal route is: {route_description}"




def get_directions(gmaps, route, locations, mode="driving"):
    """
    Fetch turn-by-turn directions for the optimal route.

    Args:
        gmaps (googlemaps.Client): Google Maps API client.
        route (list): Optimal route indices.
        locations (dict): Dictionary of place names and coordinates.
        mode (str): Transportation mode ("driving", "walking", or "transit").

    Returns:
        list: List of Google Maps directions responses for each leg.
    """
    directions = []
    for i in range(len(route) - 1):
        # Get origin and destination for the segment
        origin = locations[route[i]]
        destination = locations[route[i + 1]]

        # Format as strings for API
        origin_str = f"{origin[0]},{origin[1]}"
        destination_str = f"{destination[0]},{destination[1]}"

        # Fetch directions for the segment
        response = gmaps.directions(
            origin=origin_str,
            destination=destination_str,
            mode=mode
        )
        directions.append(response)
    return directions


def parse_directions(directions, route):
    """
    Parse directions from Google Maps API response into a user-friendly format.

    Args:
        directions (list): List of Google Maps API directions responses for each leg.
        route (list): List of location names in the optimal route.

    Returns:
        str: User-friendly directions string.
    """
    parsed_directions = []

    for i, segment in enumerate(directions):
        if not segment:
            parsed_directions.append(f"No directions available for segment {i + 1}: {route[i]} -> {route[i + 1]}")
            continue

        leg = segment[0]['legs'][0]
        start = route[i]
        end = route[i + 1]
        distance = leg['distance']['text']
        duration = leg['duration']['text']

        # Add header for the segment
        parsed_directions.append(f"\n--- From: {start} To: {end} ---\n")
        parsed_directions.append(f"Distance: {distance}, Duration: {duration}\n")

        # Add step-by-step instructions
        for step in leg['steps']:
            instruction = step['html_instructions']
            step_distance = step['distance']['text']
            step_duration = step['duration']['text']

            # Clean HTML tags from instructions
            clean_instruction = instruction.replace('<b>', '').replace('</b>', '').replace('<div style="font-size:0.9em">', ' ').replace('</div>', '')

            # Include transit-specific details if applicable
            if "transit_details" in step:
                transit = step["transit_details"]
                line_name = transit["line"]["name"]
                vehicle_type = transit["line"]["vehicle"]["type"]
                departure_stop = transit["departure_stop"]["name"]
                arrival_stop = transit["arrival_stop"]["name"]

                parsed_directions.append(
                    f"- Take {vehicle_type} ({line_name}) from {departure_stop} to {arrival_stop} ({step_distance}, {step_duration})"
                )
            else:
                parsed_directions.append(f"- {clean_instruction} ({step_distance}, {step_duration})")

    return "\n".join(parsed_directions)



starting_location_name = "Minerva Residence"
starting_location_coords = (37.792033, -122.408465)

ending_location_name = None
ending_location_coords = None

destinations = {
    "Chinatown San Francisco": (37.792597, -122.406063),
    "California Academy of Sciences": (37.76986, -122.46609),
    "Pier 39": (37.80867, -122.40982),
    "Painted Ladies": (37.77625, -122.43275),
    "Exploratorium": (37.80166, -122.39734),
    "Lombard Street": (37.80201, -122.41955),
    "Palace of Fine Arts": (37.80293, -122.44842),
    "San Francisco City Hall": (37.77927, -122.41924),
}

# Combine starting, ending, and destinations
all_locations = {starting_location_name: starting_location_coords, **destinations}
if ending_location_name:
    all_locations[ending_location_name] = ending_location_coords

place_names = list(all_locations.keys())
coordinates = list(all_locations.values())



In [4]:
# Driving mode
mode = "driving"

print(f"--- Testing Mode: {mode.upper()} ---")

# Calculate distance matrix
distance_matrix = get_distance_matrix(gmaps, coordinates, mode=mode)

# Solve TSP and extract route
tour_matrix, optimal_cost = solve_tsp(distance_matrix)
optimal_route_indices = extract_tour(tour_matrix)
optimal_route = [place_names[i] for i in optimal_route_indices]

# Print the pretty route
print(pretty_print_route(optimal_route, optimal_cost))

# Fetch and display directions
directions = get_directions(gmaps, optimal_route, all_locations, mode=mode)
parsed_directions = parse_directions(directions, optimal_route)
print("Directions:\n", parsed_directions)


--- Testing Mode: DRIVING ---
Total time: 1 hrs, 18 mins, 46 seconds
The optimal route is: Minerva Residence -> Chinatown San Francisco -> Exploratorium -> Pier 39 -> Lombard Street -> Palace of Fine Arts -> California Academy of Sciences -> Painted Ladies -> San Francisco City Hall -> Minerva Residence
Directions:
 
--- From: Minerva Residence To: Chinatown San Francisco ---

Distance: 0.3 mi, Duration: 3 mins

- Head north on Joice St toward California St (33 ft, 1 min)
- Turn right at the 1st cross street onto California St (0.1 mi, 1 min)
- Turn right onto Quincy St (354 ft, 1 min)
- Turn right onto Pine St (89 ft, 1 min)
- Turn right at the 1st cross street onto Grant Ave Destination will be on the left (404 ft, 1 min)

--- From: Chinatown San Francisco To: Exploratorium ---

Distance: 1.0 mi, Duration: 7 mins

- Head north on Grant Ave toward Sacramento St Pass by Bank of America Financial Center (on the left) (0.4 mi, 4 mins)
- Turn right after Wells Fargo Bank (on the right) (0

In [5]:
# Transit mode
mode = "walking"

print(f"--- Testing Mode: {mode.upper()} ---")

# Calculate distance matrix
distance_matrix = get_distance_matrix(gmaps, coordinates, mode=mode)

# Solve TSP and extract route
tour_matrix, optimal_cost = solve_tsp(distance_matrix)
optimal_route_indices = extract_tour(tour_matrix)
optimal_route = [place_names[i] for i in optimal_route_indices]

# Print the pretty route
print(pretty_print_route(optimal_route, optimal_cost))

# Fetch and display directions
directions = get_directions(gmaps, optimal_route, all_locations, mode=mode)
parsed_directions = parse_directions(directions, optimal_route)
print("Directions:\n", parsed_directions)

--- Testing Mode: WALKING ---
Total time: 4 hrs, 47 mins, 48 seconds
The optimal route is: Minerva Residence -> Chinatown San Francisco -> Exploratorium -> Pier 39 -> Lombard Street -> Palace of Fine Arts -> California Academy of Sciences -> Painted Ladies -> San Francisco City Hall -> Minerva Residence
Directions:
 
--- From: Minerva Residence To: Chinatown San Francisco ---

Distance: 0.2 mi, Duration: 4 mins

- Head east on California St toward Joice St (0.1 mi, 4 mins)
- Turn left onto Grant Ave Destination will be on the left (52 ft, 1 min)

--- From: Chinatown San Francisco To: Exploratorium ---

Distance: 1.1 mi, Duration: 23 mins

- Head north on Grant Ave toward Sacramento St (0.3 mi, 6 mins)
- Turn right onto Pacific Ave (0.3 mi, 7 mins)
- Turn left onto Battery St (0.2 mi, 4 mins)
- Turn right onto Green St (0.1 mi, 3 mins)
- Turn left onto The Embarcadero N (30 ft, 1 min)
- Turn right onto Pier 15 Destination will be on the right (0.1 mi, 2 mins)

--- From: Exploratorium To

In [6]:
# Transit mode with walking fallback
mode = "transit"
print(f"--- Testing Mode: {mode.upper()} ---")

# Calculate distance matrix
distance_matrix = get_distance_matrix(gmaps, coordinates, mode=mode, buffer_minutes=5)

# Solve TSP and extract route
tour_matrix, optimal_cost = solve_tsp(distance_matrix)
optimal_route_indices = extract_tour(tour_matrix)
optimal_route = [place_names[i] for i in optimal_route_indices]

# Print the pretty route
print(pretty_print_route(optimal_route, optimal_cost))

# Fetch and display directions
directions = get_directions(gmaps, optimal_route, all_locations, mode=mode)
parsed_directions = parse_directions(directions, optimal_route)
print("Directions:\n", parsed_directions)


--- Testing Mode: TRANSIT ---
Total time: 2 hrs, 41 mins, 58 seconds
The optimal route is: Minerva Residence -> Chinatown San Francisco -> Pier 39 -> Exploratorium -> San Francisco City Hall -> Painted Ladies -> California Academy of Sciences -> Palace of Fine Arts -> Lombard Street -> Minerva Residence
Directions:
 
--- From: Minerva Residence To: Chinatown San Francisco ---

Distance: 0.2 mi, Duration: 4 mins

- Walk to 607 Grant Ave, San Francisco, CA 94108, USA (0.2 mi, 4 mins)

--- From: Chinatown San Francisco To: Pier 39 ---

Distance: 1.7 mi, Duration: 21 mins

- Walk to Kearny St & California St (0.1 mi, 3 mins)
- Take BUS (Bayshore) from Kearny St & California St to Kearny St & North Point St (1.4 mi, 10 mins)
- Walk to 256 Pier 39, San Francisco, CA 94133, USA (0.2 mi, 5 mins)

--- From: Pier 39 To: Exploratorium ---

Distance: 1.1 mi, Duration: 11 mins

- Walk to Beach St & Stockton St (0.1 mi, 2 mins)
- Take TRAM (Market & Wharves) from Beach St & Stockton St to The Embarc